# DOCS:

[DOCS](https://rapidapi.com/rphrp1985/api/newsnow)

In [1]:
import os

def excluir_arquivos_em_pasta(pasta):
    # Verifica se a pasta existe
    if os.path.exists(pasta):
        # Lista os arquivos na pasta
        arquivos = os.listdir(pasta)
        
        # Itera sobre os arquivos e os exclui
        for arquivo in arquivos:
            caminho_arquivo = os.path.join(pasta, arquivo)
            if os.path.isfile(caminho_arquivo):
                os.remove(caminho_arquivo)
            else:
                print(f"{caminho_arquivo} não é um arquivo e não será excluído.")
        
        print("Arquivos excluídos com sucesso.")
    else:
        print(f"A pasta {pasta} não existe.")

In [19]:
# Substitua 'caminho_para_pasta' pelo caminho da pasta que você deseja verificar e limpar.
excluir_arquivos_em_pasta('temp_news')

from datetime import datetime, timedelta
import requests 

def getNews(url: str, query: str, page: int = 1, from_date: str = None, to_date: str = None):
    # Verifica se from_date e to_date não foram fornecidos e define-os como ontem
    if from_date is None: 
        yesterday = datetime.now() - timedelta(days=1)
        from_date = yesterday.strftime('%d/%m/%Y')
    
    if to_date is None: 
        yesterday = datetime.now() - timedelta(days=1)
        to_date = yesterday.strftime('%d/%m/%Y')
    
    # Converte as datas de string (dd/MM/yyyy) para objetos datetime
    from_date = datetime.strptime(from_date, '%d/%m/%Y')
    to_date = datetime.strptime(to_date, '%d/%m/%Y')

    # Formate as datas para o formato esperado
    from_date_str = from_date.strftime('%d/%m/%Y')
    to_date_str = to_date.strftime('%d/%m/%Y')

    payload = {
        "query": query,
        "page": page,
        "time_bounded": True,
        "from_date": from_date_str,
        "to_date": to_date_str,
        "location": "",
        "category": "",
        "source": ""
    }
    headers = {
        "content-type": "application/json",
        "X-RapidAPI-Key": os.getenv("RAPIDAPI"),
        "X-RapidAPI-Host": "newsnow.p.rapidapi.com"
    }

    response = requests.post(url, json=payload, headers=headers)
    bSuccess = True
    try:
        # response = response.json()['news']
        response = response
    except:
        bSuccess = False
        response = response
    finally:
        return bSuccess, response

Arquivos excluídos com sucesso.


In [20]:
bSuccess, _ = getNews(url = url,query = query, page = 1, from_date='01/10/2023', to_date = '21/10/2023')

print(json.dump(_.json()))

<Response [200]>


In [24]:
import json
print(json.dumps(_.json()))

{"page": 1, "news": [{"title": "Tomorrow's fossils: Endangered animal species in 1972", "body": "Tomorrow's fossils: Experimental embryology and endangered animal species in 1972 ... Francoeur appeared as the feature article in a 1972 magazine issue that took...", "date": "Jun 22, 2023", "url": "https://thebulletin.org/2023/06/tomorrows-fossils-experimental-embryology-and-endangered-animal-species-in-1972/", "source": "Bulletin of the Atomic Scientists", "image": "https://audiospace-1-u9912847.deta.app/getnewspic?id=xS1UyDwq8oPNNJrvzgCl1697943620.8857436"}, {"title": "Evaluation of an automated dish preparation system for IVF ...", "body": "... animal embryology experience. They were engaged in the dish preparation and embryo cultivation during the whole process. Working mechanism of automated...", "date": "3 weeks ago", "url": "https://www.nature.com/articles/s41598-023-43665-y", "source": "Nature", "image": "https://audiospace-1-u9912847.deta.app/getnewspic?id=g8Om4LAETb303f9rZWil169

In [10]:
import json
from dotenv import load_dotenv

load_dotenv()

url = "https://newsnow.p.rapidapi.com/newsv2"

#Physiology of the digestive tract in dogs
query = input("Qual o termo que você quer pesquisar?")

bGetNews = True

i = 1
while bGetNews:
    bSuccess, _ = getNews(url = url,query = query, page = i, from_date='01/01/2023', to_date = '09/10/2023')
    
    if not bSuccess:
        print(f"Erro: {_}")
        bGetNews = False
    
    if len(_):
        jsonData = json.dumps(_)
        with open(f'temp_news/newsnow_{str(i)}.json', 'w') as f:
            f.write(jsonData)
    else:
        bGetNews = False
    i += 1

In [12]:
bSuccess

True

In [13]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window as W

from pyspark.sql.types import StructType, StructField, StringType, IntegerType
# Configurando a sessão Spark
spark = SparkSession.builder.appName("biocast").getOrCreate()

23/10/21 23:56:57 WARN Utils: Your hostname, billy-virus resolves to a loopback address: 127.0.1.1; using 192.168.1.27 instead (on interface wlp0s20f3)
23/10/21 23:56:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/21 23:56:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
newsnow = spark.read.option("multiline","true").json("temp_news/*")
newsnow = (
    newsnow
)
newsnow.cache()
newsnow.repartition(1)

DataFrame[body: string, date: string, image: string, source: string, title: string, url: string]

In [15]:
from translate import Translator
from googletrans import Translator as Tgoogle
import re

# Função para remover caracteres não alfanuméricos
def remove_non_alphanumeric(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# Registrar a função UDF
remove_non_alphanumeric_udf = F.udf(remove_non_alphanumeric, StringType())

# Função para traduzir uma coluna
def translate_column_translate(text, target_lang='pt-br'):
    translator = Translator(key=os.getenv('MYMEMORYKEY'), to_lang=target_lang)
    return translator.translate(text)

# Função para traduzir uma coluna
def translate_column_google(text, source_lang='en', target_lang='pt_br'):
    try:
        if text is not None:
            translator = Tgoogle()
            translated = translator.translate(text, src=source_lang, dest=target_lang)
            return translated.text
    except Exception as e:
        print(f"Erro na tradução: {e}")
    return text

# Registrar a função UDF
translate_udf_google = F.udf(translate_column_google, StringType())

# Registrar a função UDF
translate_udf_translate = F.udf(translate_column_translate, StringType())

# Use a função para traduzir as colunas desejadas
newsnow_translate = (
    newsnow
    # .withColumn("body", remove_non_alphanumeric_udf(F.col("body")))
    # .withColumn("title", remove_non_alphanumeric_udf(F.col("title")))
    # .withColumn("title-pt_br-translate", F.lit(translate_udf_translate(F.col("title"))))
    # .withColumn("title-pt_br-google", F.lit(translate_udf_google(F.col("title"))))
    # .withColumn('title-pt_br', F.when(F.col('title')==F.col('title-pt_br-google'), F.col('title-pt_br-translate')).otherwise(F.col('title-pt_br-google')))
    # .withColumn("body-pt_br-translate", F.lit(translate_udf_translate(F.col("body"))))
    # .withColumn("body-pt_br-google", F.lit(translate_udf_google(F.col("body"))))
    # .withColumn('body-pt_br', F.when(F.col('body')==F.col('body-pt_br-google'), F.col('body-pt_br-translate')).otherwise(F.col('body-pt_br-google')))
    # .select(
    #     F.col('source')
    #     ,F.col('title')
    #     ,F.col('body')
    #     ,F.col('date')
    #     ,F.col('title-pt_br')
    #     ,F.col('body-pt_br')
    #     ,F.col('url')
    # )
)

# Exiba o DataFrame resultante
newsnow_translate.cache()

23/10/21 23:57:02 WARN CacheManager: Asked to cache already cached data.


DataFrame[body: string, date: string, image: string, source: string, title: string, url: string]

In [16]:
newsnow_translate.write.option('encoding', 'utf-8').partitionBy('source').mode('overwrite').parquet('./parquet_news')

In [17]:
teste = spark.read.parquet('parquet_news')
teste.sort(F.col('date').desc()).show()

+--------------------+------------+--------------------+--------------------+--------------------+--------------------+
|                body|        date|               image|               title|                 url|              source|
+--------------------+------------+--------------------+--------------------+--------------------+--------------------+
|So I needed an an...| May 9, 2023|https://audiospac...|Basic ‘toolkit’ f...|https://www.eurek...|         EurekAlert!|
|SignificanceHealt...| May 8, 2023|https://audiospac...|An unanticipated ...|https://www.pnas....|                PNAS|
|The researchers u...| May 4, 2023|https://audiospac...|Speeded-up Brillo...|https://physicswo...|       Physics World|
|Every animal and ...|May 30, 2023|https://audiospac...|Cells, Not DNA, A...|https://www.noema...|      Noema Magazine|
|New Zealand lives...|May 26, 2023|https://audiospac...|Brazil declares a...|https://www.farme...|     Farmer's Weekly|
|/Scientific commu...|May 25, 2023|https

In [ ]:
teste.filter(F.col('source')=='Nature').show(truncate=False)

+--------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------+------+
|title                                                         |body                                                                                                                                                        |date        |title-pt_br                                                                  

In [ ]:
teste.select('url').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|url                                                                                                                                                                                                                  |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|https://www.globenewswire.com/en/news-release/2023/03/08/2623170/0/en/Animal-Genetics-Market-Size-to-Hit-8-21-Billion-Globally-by-2028-Lead-by-Porcine-Segment-with-7-CAGR-during-2022-2028-The-Insight-Partners.html|
|https://english.elpais.com/science-tech/2023-09-07/the-first-human-organ-created-inside-an-animal-opens-the-door-to-manufacturing-spare

In [11]:
spark.stop()